In [22]:
# instalare componente necesare rularii
!pip install pandas
!pip install scikit-learn scipy matplotlib numpy pandas soundfile tabulate pydub ipython sounddevice
# tabulate este pentru creare tabele

# https://data-flair.training/blogs/python-mini-project-speech-emotion-recognition/ # model de cod
# pentru implementarea codului am urmat ideile din urmatoarele link-uri:
# 1. https://github.com/topics/speech-emotion-recognition
# 2. https://github.com/MiteshPuthran/Speech-Emotion-Analyzer
# 3. https://github.com/xuanjihe/speech-emotion-recognition
# 4. https://github.com/Demfier/multimodal-speech-emotion-recognition
# 5. https://github.com/hkveeranki/speech-emotion-recognition
# 6. https://github.com/x4nth055/emotion-recognition-using-speech
# https://www.researchgate.net/figure/The-SED-System-a-Real-Time-Speech-Emotion-Detection-System-based-on-Internet-of-Things_fig1_337992475 # discutii

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# type of generalized linear model (GLM) that uses a logistic function to model a binary dependent variable. It estimates the probability that a given input belongs to a particular class and then classifies it based on a threshold (supervised)
from sklearn.tree import DecisionTreeClassifier
# flowchart-like structure, where each internal node represents a feature(or attribute), each branch represents a decision rule, and each leaf node represents the outcome (supervised)
from sklearn.neural_network import MLPClassifier
# type of feedforward artificial neural network that consists of one or more layers of artificial neurons, called perceptrons. Each perceptron receives input from the previous layer and applies a non-linear activation function to it before passing the output to the next layer. The last layer of perceptrons is called the output layer and it generates the final predictions. The algorithm learns the weights of the perceptrons by minimizing the difference between the predicted output and the true output using an optimization algorithm, such as stochastic gradient descent. (supervised)
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier # multiple trees and combines them to make a decision by using "vote system" and the majority wins (supervised)
import os
import pickle
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC # find the best boundary (supervised)
from pydub import AudioSegment
from IPython.display import Audio
from IPython.display import display
import soundfile as sf
import sounddevice as sd
import sys
from scipy.io.wavfile import write
from tkinter import *

def path_exists(path):
  return os.path.exists(path)


class Output():
  def __init__(self):
    self.msg = ""

  def add(self, msg):
    if self.msg == "":
      self.msg = msg
    else:
      self.msg = self.msg + "\n" + msg

  def show(self):
    print(self.msg)

  def clear(self):
    self.msg = ""

  def have_output(self):
    if self.msg == "":
      return False
    else:
      return True


class Error():
  def __init__(self):
    self.msg = ""
    self.out = Output()

  def add_error(self, msg):
    if self.msg == "":
      self.msg = msg
    else:
      self.msg = self.msg + "\n" + msg
  
  def path_not_found(self, path):
    self.add_error(f"{path} nu exista")

  def got_error(self):
    if self.msg == "":
      return False
    else:
      return True

  def clear(self):
    self.msg = ""

  def show(self):
    print(self.msg)


out = Output()
err = Error()

In [8]:
class AudioManipulation:
  def __init__(self):
    pass

  def load(self, file_path):
    if path_exists(file_path):
      audio = AudioSegment.from_wav(file_path)
      # sr = sample rate
      _ , sr = sf.read(file_path)  
      return audio, sr
# Sample rate, sau "rata de eșantionare", reprezintă numărul de eșantioane de semnal luate pe secundă.
# Este exprimat în herți (Hz) și reprezintă numărul de câți eșantioane sunt luate pe secundă din semnalul audio.
# Rata de eșantionare afectează calitatea sunetului și frecvența maximă care poate fi înregistrată sau redată.
# Cu cât rata de eșantionare este mai mare, cu atât semnalul audio este redat mai precis.
# O rata de eșantionare mai mare va permite redarea frecvențelor mai înalte, dar va ocupa mai mult spațiu pe disc.
# Rata standard pentru aplicațiile audio digitale este de 44.1 sau 48 kHz.
    else:
      err.path_not_found(file_path)

  # STFT (Short-Time Fourier Transform) este o metodă utilizată pentru a analiza un semnal în timp și frecvență. Aceasta se face prin împărțirea semnalului în fragmente scurte de durată (numite "ferestre") și aplicarea unei transformări Fourier pe fiecare fragment. Aceasta permite analizarea semnalului în timp și frecvență simultan.
  # In acest caz, se utilizeaza functia numpy hanning pentru a obtine fereastra hanning si se prelungește audio cu câteva esantioane pentru a evita pierderea acestora la tăiere. Apoi, se utilizeaza functia fft din numpy pentru a efectua transformata Fourier rapida, si se returneaza spectrograma absoluta.
  def stft(self, audio, window_size=2048):
    audio_len = len(audio)
    window = np.hanning(window_size)
    # https://en.wikipedia.org/wiki/Hann_function
    # Operatorul // este operatorul de diviziune care returneaza catul diviziei, eliminand partea fractionara.
    audio = np.pad(audio, (window_size//2, window_size//2), mode='reflect')
    num_segments = (audio_len + window_size - 1) // window_size
    segments = np.zeros((num_segments, window_size))
    # Fereastra este o functie matematica care se aplica peste segmentul audio si are rolul de a reduce artefactele care apar in urma taierii audio-ului in segmente.
    # Aceasta reduce artefactele prin atenuarea (attenuation) marginilor segmentelor.
    # artefacte = distorsiuni sau abateri neintentionate in semnalul audio care apar in urma procesarii acestuia
    for i in range(num_segments):
        segment = audio[i*window_size:i*window_size+window_size]
        segments[i] = segment * window
    spectrogram = np.abs(np.fft.fft(segments, axis=1))
    # Aceasta linie de cod calculeaza spectrograma semnalului audio prin utilizarea Transformatei Fourier Rapide (FFT). Spectrograma este o reprezentare grafica a semnalului audio, care arata cum se distribuie energia in diferite frecvente de-a lungul timpului.
    # Fiecare coloana a spectrogram-ului reprezinta o transformata Fourier a unui segment din semnalul audio, iar fiecare linie reprezinta o frecventa specifica.
    # Se ia modulul FFT pentru a elimina componenta de faza.
    # Acesta este utilizat pentru a detecta si caracteriza anumite caracteristici ale sunetului cum ar fi tonalitatea sau timbrul.
    return spectrogram

  def piptrack(self, audio, sr):
    # calculam diferenta de faza intre semnalul original si cel intarziat
    audio_delayed = audio[1:]
    audio_diff = audio_delayed - audio[:-1]
    audio_diff = np.square(audio_diff)

    # aplicam o fereastra de Hanning
    window = np.hanning(len(audio_diff))
    audio_diff = audio_diff * window

    # calculam integrala spectrului
    audio_diff = np.cumsum(audio_diff)

    # determinam pitch-ul
    pitch = sr / np.argmin(audio_diff)

    return pitch
  # Aceasta functie calculeaza rata de variatie a frecventei vorbirii (pitch) din semnalul audio.
  # Prima etapa este calcularea diferentei de faza intre semnalul original si cel intarziat.
  # Aceasta este realizata prin scaderea elementelor din semnalul original cu elementele din semnalul intarziat.
  # Urmatorul pas este ridicarea la patrat a diferentei de faza pentru a elimina valorile negative.
  # Apoi se aplica o fereastra de Hanning pentru a reduce artefactele.
  # Se calculeaza integrala spectrului, care este utilizata pentru a determina pozitia minimului, care este inversul frecventei.
  # Rata de variatie a frecventei vorbirii (pitch) este obtinuta prin impartirea frecventei de esantionare la pozitia minimului.

In [9]:
class Preprocessing():
  def __init__(self, dir : str):
    self.dir = dir
    self.am = AudioManipulation()
    if not path_exists(dir): # verifica daca directorul exista sau nu
      err.path_not_found(dir)
    else:
      # listam fisierele din directorul dat ca parametru
      filenames = os.listdir(self.dir)

      # cream lista de tupluri formata din numele fisierelor audio si numele fisierelor txt asociate
      self.audio_txt_pairs = []
      for filename in filenames:
        if filename.endswith('.wav'):
          txt_filename = filename.replace('.wav', '.txt')
          full_path = os.path.join(self.dir, txt_filename)
          if path_exists(full_path):
            self.audio_txt_pairs.append((os.path.join(self.dir, filename), os.path.join(self.dir, txt_filename)))
          else:
            err.path_not_found(full_path)

  def preprocess_audio_file(self, audio_file, txt_file):
    if err.got_error():
      out.add("Ai de rezolvat una sau mai multe erori inainte de a lucra cu functia preprocess_audio_file")
      return [0], 0
    else: 
      # incarcam fisierul audio
      audio, sr = self.am.load(audio_file)
      audios = []
    
      if(path_exists(txt_file)):
        with open(txt_file, 'r') as f:
          lines = f.readlines()
          out.add(f"txt file: {txt_file}, nr linii: {len(lines)}")
          # impartim fisierul audio in fragmente
          for line in lines:
            start = float(line[:line.find("[")].split(' ')[0].split('\t')[0])
            end = float(line[:line.find("[")].split(' ')[0].split('\t')[1])
            fragment_audio = audio[start:end]
            out.add(f"range {start} - {end}")
            # adaugam fragmentul audio la lista de fragmente
            audios.append(fragment_audio)
        f.close()
      else:
        err.path_not_found(txt_file)
    
      return audios, sr

  def extract_features(self, audio, sr):
    if err.got_error():
      out.add("Ai de rezolvat una sau mai multe erori inainte de a lucra cu functia extract_features")
      return [0, 0, 0, 0, 0, 0]
    else: 
      # transformare din pydub audiosegment in numpy array
      audio = np.asarray(audio.get_array_of_samples(),dtype = np.float64)
      # calculam amplitudinea semnalului
      amplitudes = np.abs(audio)

      # calculam spectrul de frecventa
      spectrogram = np.abs(self.am.stft(audio))

      # calculam rata de variatie a frecventei vorbirii
      pitch = self.am.piptrack(audio, sr)
      pitch = np.array([pitch])
      pitch_change = np.diff(pitch)
      pitch_change_rate = np.mean(pitch_change)

      # calculam lungimea silabelor
      syllable_lengths = []
      for i in range(len(audio) - 1):
        if audio[i] > 0 and audio[i + 1] < 0:
          syllable_lengths.append(i + 1 - sum(syllable_lengths))

      # calculam durata pauzelor
      pause_lengths = []
      for i in range(len(audio) - 1):
        if audio[i] == 0 and audio[i + 1] != 0:
          pause_lengths.append(i + 1 - sum(pause_lengths))

      # calculam rata de respiratie
      respiration_rate = 0
      if(sum(syllable_lengths) != 0):
        respiration_rate = len(syllable_lengths) / sum(syllable_lengths)

      # returnam caracteristicile
      return [np.mean(amplitudes), np.mean(spectrogram), pitch_change_rate, np.mean(syllable_lengths), np.mean(pause_lengths), respiration_rate]

  
  def preprocesare(self):
    if err.got_error():
      out.add("Ai de rezolvat una sau mai multe erori inainte de a lucra cu functia preprocesare")
      return 0, 0
    else: 
      # obtinem listele audio_files si txt_files
      audio_files = [pair[0] for pair in self.audio_txt_pairs]
      txt_files = [pair[1] for pair in self.audio_txt_pairs]

      # citim etichetele de emotie din fisierele txt
      labels = []
      for txt_file in txt_files:
        with open(txt_file, 'r') as f:
          lines = f.readlines()
          for line in lines:
            label = line[line.find("["):].split(' ')[0].replace("[", "").replace("]", "")
            labels.append(label)

      # extragem caracteristicile din fisierele audio
      X = []

      for i in range(len(self.audio_txt_pairs)):
        audio, sr = self.preprocess_audio_file(audio_files[i], txt_files[i])
        for j in range(len(audio)):
          features = self.extract_features(audio[j], sr)
          X = np.append(X, features)
        out.add(f"(inainte de nan_to_num) {txt_files[i]}, X = {X}")

        # la intoarcerea din functia extract_features, pitch_change_rate si respiration_rate au cele mai mari sanse sa dea nan
        X = np.nan_to_num(X)
        out.add(f"(dupa nan_to_num) {txt_files[i]}, X = {X}")
    
      # transformam etichetele de emotie in forma utilizabila de model
      le = LabelEncoder()
      y = le.fit_transform(labels)

      # Elimină lista suplimentară din interiorul lui np.array() atunci când se definește X
      X = np.resize(X, (y.shape[0], 1))
      
      return X, y

In [10]:
class Models:
  def __init__(self):
    self.am = AudioManipulation()
    # creeam modelele
    self.RandomForestClassifierModel = RandomForestClassifier()
    self.SVCModel = SVC()
    self.LogisticRegressionModel = LogisticRegression(max_iter=500)
    # daca lasam default (max_iter=100 default), atunci imi dadea eroarea STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.
    # discutie: https://stackoverflow.com/questions/62658215/convergencewarning-lbfgs-failed-to-converge-status-1-stop-total-no-of-iter
    self.DecisionTreeClassifierModel = DecisionTreeClassifier()
    self.MLPClassifierModel = MLPClassifier()

  def save_models(self):
    if err.got_error():
      out.add("Ai de rezolvat una sau mai multe erori inainte de a salva modelele")
    else:
      with open("RandomForestClassifierModel.pkl", "wb") as f:
        # salvam modelul in fisier
        pickle.dump(self.RandomForestClassifierModel, f)
      out.add("RandomForestClassifierModel a fost salvat")

      with open("SVCModel.pkl", "wb") as f:
        # salvam modelul in fisier
        pickle.dump(self.SVCModel, f)
      out.add("SVCModel a fost salvat")

      with open("LogisticRegressionModel.pkl", "wb") as f:
        # salvam modelul in fisier
        pickle.dump(self.LogisticRegressionModel, f)
      out.add("LogisticRegressionModel a fost salvat")

      with open("DecisionTreeClassifierModel.pkl", "wb") as f:
        # salvam modelul in fisier
        pickle.dump(self.DecisionTreeClassifierModel, f)
      out.add("DecisionTreeClassifierModel a fost salvat")

      with open("MLPClassifierModel.pkl", "wb") as f:
        # salvam modelul in fisier
        pickle.dump(self.MLPClassifierModel, f)
      out.add("MLPClassifierModel a fost salvat")

  def load_models(self):
    with open("RandomForestClassifierModel.pkl", "rb") as f:
      # salvam modelul in fisier
      self.RandomForestClassifierModel = pickle.load(f)
    out.add("RandomForestClassifierModel a fost incarcat")

    with open("SVCModel.pkl", "rb") as f:
      # salvam modelul in fisier
      self.SVCModel = pickle.load(f)
    out.add("SVCModel a fost incarcat")

    with open("LogisticRegressionModel.pkl", "rb") as f:
      # salvam modelul in fisier
      self.LogisticRegressionModel = pickle.load(f)
    out.add("LogisticRegressionModel a fost incarcat")

    with open("DecisionTreeClassifierModel.pkl", "rb") as f:
      # salvam modelul in fisier
      self.DecisionTreeClassifierModel = pickle.load(f)
    out.add("DecisionTreeClassifierModel a fost incarcat")

    with open("MLPClassifierModel.pkl", "rb") as f:
      # salvam modelul in fisier
      self.MLPClassifierModel = pickle.load(f)
    out.add("MLPClassifierModel a fost incarcat")

  def train(self, x, y):

    if err.got_error():
      out.add("Nu poti antrena modelele daca ai erori")
    elif x.size == 0 and y.size == 0:
      err.add_error("Nu s-a realizat preprocesarea")
    else:
      self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(x, y, test_size=0.7, random_state=42) # 70% date de antrenare, 30% date de testare
      # random_state = 42 means that the split is reproducible, meaning that if the script is run again with the same random_state, the data will be split in the same way
      out.add(f"X_train: {self.X_train}")
      out.add(f"y_train: {self.y_train}")
      out.add(f"X_test: {self.X_test}")
      out.add(f"y_test: {self.y_test}")
      self.RandomForestClassifierModel.fit(self.X_train, self.y_train)
      self.SVCModel.fit(self.X_train, self.y_train)
      self.LogisticRegressionModel.fit(self.X_train, self.y_train)
      self.DecisionTreeClassifierModel.fit(self.X_train, self.y_train)
      self.MLPClassifierModel.fit(self.X_train, self.y_train)
      out.add("Modelele au fost antrenate")

  def accuracy_score(self, y_true, y_pred):
    num_correct = 0
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i]:
            num_correct += 1
    return num_correct / len(y_true)

  def models_accuracy(self):
    if err.got_error():
      out.add("Ai una sau mai multe erori de rezolvat")
    else:
      # evaluam acuratetea modelului pe datele de testare
      self.y_pred_RandomForestClassifier = self.RandomForestClassifierModel.predict(self.X_test)
      self.y_pred_SVC = self.SVCModel.predict(self.X_test)
      self.y_pred_LogisticRegression = self.LogisticRegressionModel.predict(self.X_test)
      self.y_pred_DecisionTreeClassifier = self.DecisionTreeClassifierModel.predict(self.X_test)
      self.y_pred_MLPClassifier = self.MLPClassifierModel.predict(self.X_test)

      self.accuracy_RandomForestClassifier = self.accuracy_score(self.y_test, self.y_pred_RandomForestClassifier)
      self.accuracy_SVC = self.accuracy_score(self.y_test, self.y_pred_SVC)
      self.accuracy_LogisticRegression = self.accuracy_score(self.y_test, self.y_pred_LogisticRegression)
      self.accuracy_DecisionTreeClassifier = self.accuracy_score(self.y_test, self.y_pred_DecisionTreeClassifier)
      self.accuracy_MLPClassifier = self.accuracy_score(self.y_test, self.y_pred_MLPClassifier)

      out.add(f'Acuratetea modelului RandomForestClassifier este: {self.accuracy_RandomForestClassifier:.2f}')
      out.add(f'Acuratetea modelului SVC este: {self.accuracy_SVC:.2f}')
      out.add(f'Acuratetea modelului LogisticRegression este: {self.accuracy_LogisticRegression:.2f}')
      out.add(f'Acuratetea modelului DecisionTreeClassifier este: {self.accuracy_DecisionTreeClassifier:.2f}')
      out.add(f'Acuratetea modelului MLPClassifier este: {self.accuracy_MLPClassifier:.2f}')

  def create_table(self):
    from tabulate import tabulate

    #create data
    data = [["RandomForestClassifier Model", self.accuracy_RandomForestClassifier], 
            ["SVC Model", self.accuracy_SVC], 
            ["LogisticRegression Model", self.accuracy_LogisticRegression], 
            ["DecisionTreeClassifier Model", self.accuracy_DecisionTreeClassifier],
            ["MLPClassifier Model", self.accuracy_MLPClassifier]]
  
    #define header names
    col_names = ["Models", "Accuracy"]
  
    #display table
    out.add("\n")
    out.add(tabulate(data, headers=col_names))


  def use_models(self, audio_filename, p : Preprocessing):
    audio, sr = self.am.load(audio_filename)

    # extrageti caracteristicile fisierului audio
    X = p.extract_features(audio, sr)

    # preprocesati caracteristicile
    X = StandardScaler().fit_transform(X.reshape(1, -1))

    # folositi modelele pentru a face predictia
    self.prediction_RandomForestClassifierModel = self.RandomForestClassifierModel.predict(X)
    self.prediction_SVCModel = self.SVCModel.predict(X)
    self.prediction_LogisticRegressionModel = self.LogisticRegressionModel.predict(X)
    self.prediction_DecisionTreeClassifierModel = self.DecisionTreeClassifierModel.predict(X)
    self.prediction_MLPClassifierModel = self.MLPClassifierModel.predict(X)

    # transformati rezultatul predictiei in eticheta de emotie
    label_encoder = LabelEncoder()
    emotion_labels = label_encoder.fit_transform(["A", "B", "D", "F", "H", "I", "S", "N"])
    self.emotion_RandomForestClassifierModel = emotion_labels[self.prediction_RandomForestClassifierModel[0]]
    self.emotion_SVCModel = emotion_labels[self.prediction_SVCModel[0]]
    self.emotion_LogisticRegressionModel = emotion_labels[self.prediction_LogisticRegressionModel[0]]
    self.emotion_DecisionTreeClassifierModel = emotion_labels[self.prediction_DecisionTreeClassifierModel[0]]
    self.emotion_MLPClassifierModel = emotion_labels[self.prediction_MLPClassifierModel[0]]

    out.add(f"Predictia lui RandomForestClassifierModel este {self.emotion_RandomForestClassifierModel}")
    out.add(f"Predictia lui SVCModel este {self.emotion_SVCModel}")
    out.add(f"Predictia lui LogisticRegressionModel este {self.emotion_LogisticRegressionModel}")
    out.add(f"Predictia lui DecisionTreeClassifierModel este {self.emotion_DecisionTreeClassifierModel}")
    out.add(f"Predictia lui MLPClassifierModel este {self.emotion_MLPClassifierModel}")


In [11]:
class Record:
# pe google colab nu va merge PortAudioError: Error querying device -1
# nu exista un device de inregistrare
  def __init__(self, filename):
    fs = 44100  # frecventa de esantionare
    seconds = 3  # durata inregistrarii

    myrecording = sd.rec(int(seconds * fs), samplerate=fs, channels=2)
    sd.wait()  # Wait until recording is finished
    write(filename, fs, myrecording)  # Save as WAV file 

In [20]:
class App:
  def __init__(self):
    self.version_error = False
    self.trained_models = False
    self.loaded_models = False
    self.from_recorded = False
    self.recorded_filename = "output.wav"
    if sys.version_info < (3, 6):
      self.version_error = True
# Cele mai multe dintre librariile folosite de mine sunt compatibile cu Python 3.x, cu unele necesitand versiunea 3.6 sau 3.7
# Sounddevice, de exemplu, necesită Python 3.6 sau mai recent, iar soundfile necesită Python 3.5 sau mai recent
# Asa ca, pentru ca cineva sa poata rula aplicatia mea, trebuie sa aiba cel putin Python 3.6 instalat

  def train(self, dir):
    if self.version_error:
      err.add_error("Trebuie sa ai minim python 3.6 inainte sa poti rula aceasta aplicatie")
    else:
      # realizare preprocesare pe baza directorului dir
      self.p = Preprocessing(dir)
      # extragere date
      # x este pentru fisierele audio
      # y este pentru emotii
      self.x, self.y = self.p.preprocesare()
      # creare model (in functia Model vei vedea ca sunt 5 modele de fapt)
      self.m = Models()
      # antrenare model
      self.m.train(self.x, self.y)
      # afisare acuratete modele
      self.m.models_accuracy()
      # creare tabel pentru a putea vedea detaliile mai bine
      self.m.create_table()
      self.trained_models = True

  def save_models(self):
    self.m.save_models()

  def load_models(self):
    self.m.load_models()
  
  def use(self, audio_filename):
    if self.version_error:
      err.add_error("Trebuie sa ai minim python 3.6 inainte sa poti rula aceasta aplicatie")
    else:
      self.m.use_models(audio_filename, self.p)

  def show_output(self):
    if err.got_error():
      print("erori:")
      err.show()
      err.clear()
    if out.have_output():
      print("output:")
      out.show()
      out.clear()

  def start(self, choice):
    if self.version_error:
      err.add_error("Trebuie sa ai minim python 3.6 inainte sa poti rula aceasta aplicatie")
    else:
      if choice == "antrenare":
        # nu o sa mearga pe colab
        Tk().withdraw() # prevents an empty tkinter window from appearing
        dir = filedialog.askdirectory()
        self.train(dir)
      elif choice == "salvare":
        self.save_models()
      elif choice == "incarcare":
        self.load_models()
      elif choice == "folosire":
        if not self.trained_models and not self.load_models:
          err.add("Nu poti folosi modelele daca nu au fost antrenate sau incarcate inainte")
        else:
          filename = ""
          if self.from_recorded:
            filename = self.recorded_filename
          while not filename.endswith(".wav"):
            Tk().withdraw() # prevents an empty tkinter window from appearing
            audio_file = filedialog.askopenfilename()
            if not filename.endswith(".wav"):
              print("Fisierul nu este de tip wav")
          self.from_recorded = False
          self.use(filename)
      elif choice == "inregistrare":
        r = Record(self.recorded_filename)
        self.recorded = True
      else:
        err.add("Alegere neidentificata")


In [21]:
# forma adnotarilor: [eticheta emotie] [etichete fundal] [eticheta personaj] text
# eticheta emotie poate avea urmatoarele valori:
#   A = anger
#   B = boredeom
#   D = disgust
#   F = fear or anxiety
#   H = happiness
#   I = irritation or nervousness
#   N = neutral
#   S = sadness
# eticheta fundal poate avea urmatoarele valori:
#   zgomot - pt zona de zgomot
#   zgomot fundal - pt zgomotul de fundal
#   voci - pt voci nedeslusite
#   voci fundal - pentru vocile care se aud pe fundal
#   tipete fundal - pentru vocile de pe fundal care dau impresia de spaima, frica
#   muzica - pt zona de muzica
#   muzica fundal - pt muzica care se aude pe fundal
# eticheta personaj poate avea urmatoarele valori:#
#   apelant - cel care suna la 112
#   operator - cel care raspunde la telefon
#   voci - cel care se aude nu este niciunul dintre cele doua personaje
# text - zona de text reprezentand ceea ce zice personajul


dir = "train"

app = App()
choice = input("1. Antrenare modele\n2. Salvare modele\n3. Incarcare modele\n4. Folosire modele\n5. Inregistrare voce\n Alegerea ta e: ")
choice = int(choice)
alegere = ""
if choice == 1:
  alegere = "antrenare"
elif choice == 2:
  alegere = "salvare"
elif choice == 3:
  alegere = "incarcare"
elif choice == 4:
  alegere = "folosire"
else:
  alegere = "inregistrare"

app.start(alegere)
app.show_output()

1. Antrenare modele
2. Salvare modele
3. Incarcare modele
4. Folosire modele
5. Inregistrare voce
 Alegerea ta e: 1


C:\Users\User\AppData\Local\Temp\ipykernel_12144\3210080205.py:57: RuntimeWarning: divide by zero encountered in longlong_scalars
  pitch = sr / np.argmin(audio_diff)
C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


output:
txt file: C:/Users/User/Desktop/Licenta/train\_Dialogul 1 - carte de identitate - varianta dezgust IulianLuciana_Florina_Paul.txt, nr linii: 27
range 1.706667 - 3.041814
range 3.866122 - 5.491519
range 6.130068 - 7.534875
range 7.534875 - 8.904853
range 11.029478 - 12.329796
range 14.686621 - 17.345306
range 18.111565 - 21.420408
range 21.838367 - 27.980045
range 29.303583 - 31.056689
range 31.045079 - 36.641088
range 35.81678 - 48.460045
range 49.156644 - 60.847891
range 60.940771 - 67.860317
range 67.848707 - 78.297687
range 79.226485 - 87.632109
range 87.852698 - 91.823311
range 91.91619 - 97.779229
range 97.29161 - 100.112834
range 100.263764 - 109.226667
range 109.667846 - 121.835102
range 121.115283 - 129.288707
range 129.277098 - 143.55737
range 143.56898 - 146.007075
range 146.448254 - 147.179683
range 147.690522 - 151.254785
range 150.256327 - 157.454512
range 157.466122 - 158.94059
(inainte de nan_to_num) C:/Users/User/Desktop/Licenta/train\_Dialogul 1 - carte de iden

In [16]:
if sys.version_info < (3, 6):
    print("Versiunea ta de Python este mai veche decat 3.6. Te rugam sa actualizezi pentru a putea utiliza acest script.")
else:
    print("all good")

all good
